In [ ]:
import MDAnalysis as mda
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from MDAnalysis.analysis import distances

In [ ]:
u = mda.Universe('./eq.pdb', './pulling.xtc')

In [ ]:
print(u)

In [ ]:
epj_ag = u.select_atoms('resname EPJ')
phen_ag = u.select_atoms('resname PHEN')
dists = []
for ts in u.trajectory:
    dists.append(distances.distance_array(epj_ag.positions,
                                    phen_ag.positions,
                                    box=u.dimensions).mean())

In [ ]:
plt.plot(dists)
plt.xlabel('frame')
plt.ylabel('distance (angstrom)')

In [ ]:
# create windows
distances_windows = np.linspace(4, 16, 40)
#print(distances_windows)

for i, d in enumerate(distances_windows):
    os.makedirs(f'./us/run_{i}', exist_ok=True)
    # get the frame number where the distance is cloest to d
    frame = np.argmin(np.abs(dists - d))
    print(f'run {i}, d: {d}, frame: {frame}, distance: {dists[frame]}')
    print('difference', np.abs(dists[frame] - d))
    # write the frame to a pdb file
    u.trajectory[frame]
    u.atoms.write(f'./us/run_{i}/start.pdb')

In [ ]:
# generate the mdp file
for i, dist in enumerate(distances_windows):
    with open('./mdp/pulling_eq.mdp', 'r') as f:
        mdp_lines = f.readlines()
        # change init_value to the distance in the window
        for j, line in enumerate(mdp_lines):
            if 'init_value' in line:
                # should be in nm
                mdp_lines[j] = f'pull_coord1_init = {dist/10:.3f}\n'
    with open(f'./us/run_{i}/pulling_eq.mdp', 'w') as f:
        f.writelines(mdp_lines)
    with open('./mdp/pulling_md.mdp', 'r') as f:
        mdp_lines = f.readlines()
        # change init_value to the distance in the window
        for j, line in enumerate(mdp_lines):
            if 'init_value' in line:
                # should be in nm
                mdp_lines[j] = f'pull_coord1_init = {dist/10:.3f}\n'
    with open(f'./us/run_{i}/pulling_md.mdp', 'w') as f:
        f.writelines(mdp_lines)

## Analysis equilibration

In [ ]:
u_eq = []

for i, dist in enumerate(distances_windows):
    u_eq.append(mda.Universe(f'./us/run_{i}/start.pdb',
                             f'./us/run_{i}/eq/pulling_eq.xtc'))

In [ ]:
dist_eq = []

for u in u_eq:
    epj_ag = u.select_atoms('resname EPJ')
    phen_ag = u.select_atoms('resname PHEN')
    dists = []
    for ts in u.trajectory[:10]:
        # not exactly the same as the distance in the window
        dists.append(distances.distance_array(epj_ag.positions,
                                              phen_ag.positions,
                                              box=u.dimensions).mean())
    dist_eq.append(dists)

In [ ]:
# plot the distribution of the distances
fig, ax = plt.subplots()
ax.plot(np.asarray(dist_eq).mean(axis=1), label='mean', color='black')
ax.plot(np.asarray(dist_eq).min(axis=1), label='min', color='red')
ax.plot(np.asarray(dist_eq).max(axis=1), label='max', color='blue')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.set_style('whitegrid')
sns.set_palette('Reds',n_colors=40)
for dist, d in zip(dist_eq, distances_windows):
    sns.histplot(dist, label=f'{d:.2f} A', ax=ax, stat='density',
                 fill=False, common_norm=False)
#    ax.vlines(d, 0, 6, linestyle='--', color='black')
ax.set_xlabel('distance (angstrom)')
ax.set_ylabel('probability density')
ax.legend(ncol=4, bbox_to_anchor=(1, 1.1))

## Analysis Umbrella sampling

In [ ]:
import glob
import os
import numpy as np
from fe_gmx.utils.utils import natural_keys
from fe_gmx.utils.xvg import XVG
from pyemma.thermo import wham

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pullx_xvg_files = []
pull_tprs = []

for folder in glob.glob('us/run_*/pullx.xvg'):
    pullx_xvg_files.append(folder)

for folder in glob.glob('us/run_*/pulling_md.tpr'):
    pull_tprs.append(folder)

pullx_xvg_files.sort(key=natural_keys)
pull_tprs.sort(key=natural_keys)

with open('./pullx.dat', 'w') as f:
    for pullx_xvg_file in pullx_xvg_files:
        f.write(f'{pullx_xvg_file}\n')

with open('./tpr.dat', 'w') as f:
    for pull_tpr in pull_tprs:
        f.write(f'{pull_tpr}\n')

In [ ]:
pullx_xvgs = [XVG(f) for f in pullx_xvg_files]

In [ ]:
max_pullx = np.max([xvg.data['dim_2'].values.max() for xvg in pullx_xvgs])
min_pullx = np.min([xvg.data['dim_2'].values.min() for xvg in pullx_xvgs])
print(max_pullx, min_pullx)

In [ ]:
# discretize x into 100 bins
x_space = np.linspace(0.33, 1.74, 200)
# get the closest index of the x_space fro each arr of pullx
d_trajs = []
pullx_trajs = []
for i, xvg in enumerate(pullx_xvgs):
    pullx_arr = xvg.data['dim_2'].values
    d_trajs.append(np.argmin(np.abs(x_space[:, np.newaxis] - pullx_arr), axis=0))
    pullx_trajs.append(pullx_arr)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.histplot(np.concatenate(dtrajs), bins=100, ax=ax, stat='density',
             fill=False, common_norm=False)

In [ ]:
from pyemma.thermo import estimate_umbrella_sampling

In [ ]:
us_centers = list(np.linspace(4, 16, 40) / 10)
us_force_constants = [1000] * len(us_centers)
us_trajs = pullx_trajs
us_dtrajs = d_trajs
wham = estimate_umbrella_sampling(us_trajs=us_trajs,
                                  us_dtrajs=us_dtrajs,
                                  us_centers=us_centers,
                                  us_force_constants=us_force_constants,
                                  estimator='wham', lag=5,
                                  kT=2.494339, maxiter=100000,)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
fes = wham.f * 2.494339
fes = fes - fes[0]
sns.lineplot(x_space, fes, ax=ax, color='black')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.histplot(np.concatenate(pullx_trajs), bins=100, ax=ax, stat='probability',
             fill=False, common_norm=False)
            
sns.lineplot(x_space, wham.pi, ax=ax, color='black')